In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import keras
path='/Users/marcosaletta/workingDir/AWSTest/device_failure_new.csv'

Using TensorFlow backend.


In [2]:
df=pd.read_csv(path)

In [3]:
df.dtypes
df.groupby('failure').count()
df.attribute1.value_counts() #Numeric 5
df['diff7-8'] = df.attribute7 - df.attribute8
#df.describe()
df.describe()
#df.columns
#df.attribute9.hist(bins=10)
##df.loc[df.attribute9>0,['failure','attribute9']]

## First cleaning phase

looking at the date the features 7 and 8 seems to have the same distribution of value.

In onder to check if the bring the same values i make the difference between the columns

In [4]:
df['diff7-8'] = df.attribute7 - df.attribute8
df[['diff7-8']].describe()

In [5]:
df.drop(columns=['attribute7','diff7-8'],inplace=True)

In [6]:
df.attribute2.hist(bins=10)
df.loc[df.attribute2==0].shape

(118110, 11)

In [7]:
#Normalizing columns with more values
from sklearn import preprocessing

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
for i in [1,5,6]:# range(1,10):
    cur_col = 'attribute%s'%i
    print('working on %s'%cur_col)
    df[[cur_col]] = min_max_scaler.fit_transform(df[[cur_col]])
#df = pandas.DataFrame(x_scaled)
df.describe()

working on attribute1
working on attribute5
working on attribute6


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [8]:
df.device.value_counts()
dfDevice = pd.DataFrame(df.loc[df.failure==1].device)
#df.loc[df.failure==1].groupby('device').date.nunique()
dfFailed = pd.merge(dfDevice,df,on='device',how='inner')
dfFailed.groupby('device').date.nunique().describe()
#type(dfDevice)

count    106.000000
mean     101.066038
std       76.009306
min        5.000000
25%       26.500000
50%       92.000000
75%      148.000000
max      299.000000
Name: date, dtype: float64

In [9]:
#Check if the failure is always the last point int the time series
dfFailed.groupby('device').max()[['date','failure']]

## Check of failure distrubution

check if the distribution of the target variabile is balanced or not

In [12]:
dfDistro = df.groupby('failure').count()[['device']]
dfDistro

## Balancing the class

In [13]:
for i in range(1,10):
    cur_col = 'attribute%s'%i
    for lag in range(1,4):
        curNewCol = "%s_lag%s"%(cur_col,lag)
        df[[curNewCol]] = df[['attribute1']].diff(lag)
df.fillna(0,inplace=True)
df.head()

In [14]:
# Separate majority and minority classes
df_majority = df[df.failure==0]
df_minority = df[df.failure==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=df_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.failure.value_counts()

1    124388
0    124388
Name: failure, dtype: int64

In [ ]:
listNotFailed = df.loc[df.failure==0,'device']
df.set_index('device').loc[listNotFailed]

In [27]:
# Y = df['failure']
# X = df.drop(columns=['failure','device'])

Y = df_upsampled['failure']
X = df_upsampled.drop(columns=['failure','device'])

# dfDate=df.sort_values(by=['device','date'], ascending=True)
# dfDate['day']=[i for i in range(0,dfDate.shape[0])]
# Y = dfDate['failure']
# X = dfDate.drop(columns=['failure','device'])
# listFailed = df.loc[df.failure==1,'device']
# df.set_index('device').loc[listFailed]
df.shape

(124494, 38)

## Model Creation

In [32]:
# from sklearn.model_selection import StratifiedKFold
# sfk = StratifiedKFold(n_splits=2)

learning_rate = 0.001
num_steps = 30
batch_size = 100

# Network Parameters
n_hidden_2 = 10 
num_input = 30
num_classes = 1


learning_rate = 0.001
num_steps = 100
batch_size = 100

# Network Parameters
n_hidden_2 = 10 
num_input = 30
num_classes = 1

metrics = ['accuracy','mse']

class CreateNN:
    def __init__(self,**kargs):
        self.activationFun = 'relu'
        self.X = kargs['x']
        self.Y = kargs['y']
        self.skf = kargs['s'] 
        self.numFeatures = self.X.shape[1]
        self.n_hidden_1 = int(self.numFeatures / 2)
              
    def modelDefinition(self):
        self.model = Sequential()
        self.model.add(Dense(self.numFeatures+1, input_dim=self.numFeatures,
                             activation=self.activationFun))
        self.model.add(Dense(self.n_hidden_1,activation=self.activationFun))
        self.model.add(Dense(n_hidden_2,activation=self.activationFun))
        self.model.add(Dense(num_classes,activation='sigmoid'))
        
    def modelDefinitionRecurrent(self):
        self.model = Sequential()
        self.model.add(Dense(self.numFeatures+1, input_dim=self.numFeatures,
                             activation=self.activationFun))
        keras.layers.SimpleRNN(1) #activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)
        self.model.add(Dense(num_classes,activation='sigmoid'))
             
    def modelCompile(self):
        sgd = SGD(lr=learning_rate)
        self.model.compile(loss='binary_crossentropy', optimizer=sgd,metrics=metrics)   
        
    def modelTrain(self):
        print(np.unique(self.Y))
        class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(self.Y),
                                                 self.Y)
        print('----------------->',class_weights)
#         class_weight = {1:1,0:weightClasses}
        history = self.model.fit(self.X, self.Y, epochs=num_steps,
                                 batch_size=batch_size,class_weight=class_weights)
        return history,self.model
    
    def modelTrainCross(self):
        totalScores = list()
        print(np.unique(self.Y))
        class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(self.Y),
                                                 self.Y)
        print('----------------->',class_weights)
        for train, test in self.skf.split(self.X,self.Y):
#         class_weight = {1:1,0:weightClasses}
            history = self.model.fit(self.X.iloc[train], self.Y.iloc[train], epochs=num_steps,
                                 batch_size=batch_size,class_weight=class_weights)
            X_test = self.X.iloc[test]
            Y_test = self.Y.iloc[test]
            score =self.model.evaluate(X_test,Y_test)
            pred = self.model.predict(X_test).round()
            print('--------------------------Partial result: ',score)
            print('1 in test %i, 1 in pred: %i'%(np.count_nonzero(Y_test),np.count_nonzero(pred)))
            tn, fp, fn, tp = confusion_matrix(Y_test,pred).ravel()
            print('tn: %i, fp:%i, fn=%i,tp:%i'%(tn, fp, fn, tp))
            totalScores.append(score[1])
        return history,self.model, totalScores

In [33]:
skf = StratifiedKFold(n_splits=5)
cnn=CreateNN(x=X.drop(columns=['date']),y=Y,s = skf)
cnn.modelDefinition()
#cnn.modelDefinitionRecurrent()
cnn.modelCompile()
#history,model = cnn.modelTrain()
history, model,totalScores = cnn.modelTrainCross()

[0 1]
-----------------> [1. 1.]
Epoch 1/100
199020/199020 [==============================] - 3s 13us/step - loss: 0.8431 - acc: 0.7670 - mean_squared_error: 0.1819
Epoch 2/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.7935 - acc: 0.8150 - mean_squared_error: 0.1567
Epoch 3/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.7571 - acc: 0.8221 - mean_squared_error: 0.1497
Epoch 4/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.7824 - acc: 0.8234 - mean_squared_error: 0.1432 1s - loss: 0.8
Epoch 5/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.7387 - acc: 0.8168 - mean_squared_error: 0.1395
Epoch 6/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.7278 - acc: 0.8207 - mean_squared_error: 0.1369 0s - loss: 0.7282 - acc: 0.8209 - mean_squared_error
Epoch 7/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.7187 - acc: 0.8234 - me

199020/199020 [==============================] - 3s 14us/step - loss: 0.3627 - acc: 0.8758 - mean_squared_error: 0.0900
Epoch 61/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.3596 - acc: 0.8795 - mean_squared_error: 0.0890
Epoch 62/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.3574 - acc: 0.8808 - mean_squared_error: 0.0882
Epoch 63/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.3836 - acc: 0.8795 - mean_squared_error: 0.0889
Epoch 64/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.4311 - acc: 0.8769 - mean_squared_error: 0.0907
Epoch 65/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.4284 - acc: 0.8778 - mean_squared_error: 0.0898
Epoch 66/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.4259 - acc: 0.8788 - mean_squared_error: 0.0889
Epoch 67/100
199020/199020 [==============================] - 2s 12us/step - loss:

199020/199020 [==============================] - 2s 12us/step - loss: 0.3419 - acc: 0.9397 - mean_squared_error: 0.0590
Epoch 16/100
199020/199020 [==============================] - 3s 13us/step - loss: 0.3529 - acc: 0.9371 - mean_squared_error: 0.0609
Epoch 17/100
199020/199020 [==============================] - 3s 14us/step - loss: 0.3409 - acc: 0.9419 - mean_squared_error: 0.0583 2s - loss: 0.3384 - acc: 0.9
Epoch 18/100
199020/199020 [==============================] - 3s 13us/step - loss: 0.3385 - acc: 0.9431 - mean_squared_error: 0.0576 
Epoch 19/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.3365 - acc: 0.9435 - mean_squared_error: 0.0570
Epoch 20/100
199020/199020 [==============================] - 2s 12us/step - loss: 0.3358 - acc: 0.9436 - mean_squared_error: 0.0568
Epoch 21/100
199020/199020 [==============================] - 2s 11us/step - loss: 0.3351 - acc: 0.9444 - mean_squared_error: 0.0565
Epoch 22/100
199020/199020 [========================

KeyboardInterrupt: 

In [22]:
np.mean(totalScores)

0.8043058139164605